In [36]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier as knn
import seaborn as sns
import random as rnd
import csv
import random
import math
import operator
from pprint import pprint 

In [37]:
train_data = pd.read_csv("C:\\Users\\Aklesh Mishra\\q2\\train.csv",header=None)
train_data = train_data.rename(columns={0:"class"})

# Replace missing values by mode of the column with the missing values
miscol_mode = train_data.mode()[11]
train_data = train_data.fillna({11:miscol_mode})

'''
# Dr
drop_index = []
for i in range(len(train_data)):
    if train_data.iloc[i,11] == '?':
        drop_index.append(i)
df = train_data
for i in drop_index:
    df = df.drop(train_data.index[i])
train_data = df
'''
test_df = pd.read_csv("C:\\Users\\Aklesh Mishra\\q2\\test.csv",header=None)
miscol_mode = test_df.mode()[10]
test_df = test_df.fillna({10:miscol_mode})
test_label_df = pd.read_csv("C:\\Users\\Aklesh Mishra\\q2\\test_labels.csv",header=None)
test_label_df = test_label_df.rename(columns={0:"class"})

"\nres = []\ntest_data = test_df.values\nfor x in range(len(test_df.columns)):\n    cnt=0\n    for i in test_data[:,x]:\n        if i == '?':\n            cnt+=1\n    res.append(cnt)\n#print(test_data[6,:])\nprint(res[10])\n#print(len(train_data))\n"

# Scikit Learn

In [35]:
train = train_data.values
X_train = train[:,1:]
Y_train = train[:,0]
X_test = test_df.values
Y_test = test_label_df.values
k=4
knn = neighbors.KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train,Y_train)
Y_pred = knn.predict(X_test)
scores = metrics.accracy_score(Y_test,Y_pred)
print('KNN score: %f' % score)

KNN score: 38.788800


In [481]:
def train_valid_split(train_data,valid_size):
    if isinstance(valid_size,float):
        valid_size=round(valid_size * len(train_data))
    indices = train_data.index.tolist()
    valid_indices = random.sample(population=indices,k=valid_size)
    valid_df = train_data.loc[valid_indices]
    train_df = train_data.drop(valid_indices)
    return train_df,valid_df

In [482]:
random.seed(0)
train_df,valid_df = train_valid_split(train_data,valid_size=0.3)
train_dataset = train_df.values
valid_dataset = valid_df.values

In [483]:
def get_performance_measure(data,predicted):
    correct = 0
    TP = TN = FP = FN = 0
    
    for x in range(len(data)):
        if data[x,:][0] == 'e':
            if predicted[x] == 'e':
                TP += 1
            else:
                FN += 1
        else:
            if predicted[x] == 'e':
                FP += 1
            else:
                TN += 1
    
    precision = (TP/(float)(TP + FP))
    recall = (TP/(float)(TP + FN))
    F1_score = 2/(float)((1/precision)+(1/recall))   
    
    for x in range(len(data)):
        if data[x,:][0] == predicted[x]:
            correct += 1
    
    accuracy = (correct/(float)(len(data))) * 100.0
    
    return precision, recall, F1_score, accuracy

def getresponse(neighbours):
    classvotes = {}
    Max = 0
    for x in range(len(neighbours)):
        if neighbours[x][0] in classvotes:
            classvotes[neighbours[x][0]]+=1
        else:
            classvotes[neighbours[x][0]]=1
        '''
        if classvotes[neighbours[x][0]] > Max:
            Max = classvotes[neighbours[x][0]]
            label = neighbours[x][0]
        '''
    sortedvotes = sorted(classvotes.items(),key=operator.itemgetter(1),reverse=True)
    return sortedvotes[0][0]
    #return label

# Baseline Algorithms

## Random Guessing

In [484]:
def random_algorithm(train, test):
    output_values = [row[0] for row in train]
    unique = list(set(output_values))
    predicted = list()
    for row in test:
        index = random.randrange(len(unique))
        predicted.append(unique[index])
    return predicted

## Majority Voting

In [485]:
def majority_voting_algorithm(train, test):
    output_values = [row[-1] for row in train]
    prediction = max(set(output_values), key=output_values.count)
    predicted = [prediction for i in range(len(test))]
    return predicted

## Dropping the missing values of the columns with missing values

In [463]:
predicted = random_algorithm(train_dataset,test_dataset)
precision, recall, F1_score, accuracy = get_performance_measure(test_label_data,prediction)
print("Precision=%f" % precision, "Recall=%f" %recall, "F1-score=%f" %F1_score,"accuracy=%f" %accuracy)

Precision=0.254279 Recall=0.205941 F1-score=0.227571 accuracy=29.400000


In [464]:
predicted = majority_voting_algorithm(train_dataset,test_dataset)
precision, recall, F1_score, accuracy = get_performance_measure(test_label_data,prediction)
print("Precision=%f" % precision, "Recall=%f" %recall, "F1-score=%f" %F1_score,"accuracy=%f" %accuracy)

Precision=0.254279 Recall=0.205941 F1-score=0.227571 accuracy=29.400000


## Replacing missing values with mode of the column with missing values

In [486]:
train_dataset = train_df.values
valid_dataset = valid_df.values
predicted = random_algorithm(train_dataset,test_dataset)
precision, recall, F1_score, accuracy = get_performance_measure(test_label_data,prediction)
print("Precision=%f" % precision, "Recall=%f" %recall, "F1-score=%f" %F1_score,"accuracy=%f" %accuracy)

Precision=0.286017 Recall=0.267327 F1-score=0.276356 accuracy=29.300000


In [487]:
predicted = majority_voting_algorithm(train_dataset,test_dataset)
precision, recall, F1_score, accuracy = get_performance_measure(test_label_data,prediction)
print("Precision=%f" % precision, "Recall=%f" %recall, "F1-score=%f" %F1_score,"accuracy=%f" %accuracy)

Precision=0.286017 Recall=0.267327 F1-score=0.276356 accuracy=29.300000


# Hyperparameter Tuning

In [488]:
def proximity_measure(train_data,Instance,length):
    matches=0
    for x in range(length):
        if train_data[x] == Instance[x]:
            matches += 1
    distance = (matches/(float)(len(train_data)));
    return distance;

def getneighbours(train_data,Instance,Isparametertunning,k):
    distances = []
    length = len(Instance)-1
    for x in range(len(train_data)):
        if Isparametertunning == True:
            dist = proximity_measure(train_data[x,1:],Instance[1:],length)
        else:
            dist = proximity_measure(train_data[x,1:],Instance,length)

        distances.append((train_data[x,:],dist))
    distances.sort(key=operator.itemgetter(1))
    neighbours = []
    for x in range(k):
        neighbours.append(distances[x][0])
    return neighbours

In [470]:
grid_search = {"K": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-score": []}
train_dataset = train_df.values
valid_dataset = valid_df.values
for k in range(2,20):
    prediction = []
    for x in range(len(valid_dataset)):
        neighbours = getneighbours(train_dataset,valid_dataset[x,:],True,k)
        result = getresponse(neighbours)
        prediction.append(result)
        #print('> predicted: '+ repr(result) + ', actual:' + repr(valid_dataset[x,:][0]))
    precision, recall, F1_score, accuracy = get_performance_measure(valid_dataset,prediction)
    grid_search["K"].append(k)
    grid_search["Accuracy"].append(accuracy)
    grid_search["Precision"].append(precision)
    grid_search["Recall"].append(recall)
    grid_search["F1-score"].append(F1_score)
    
grid_search = pd.DataFrame(grid_search)
grid_search.sort_values("Accuracy", ascending=False).head()

,K,Accuracy,Precision,Recall,F1-score
17,19,34.707904,0.257703,0.231738,0.244032
15,17,32.989691,0.226744,0.196474,0.210526
14,16,32.760596,0.222222,0.191436,0.205683
16,18,32.646048,0.221574,0.191436,0.205405
13,15,32.416953,0.223496,0.196474,0.209115


## Replacing missing values by mode of the column with missing values

In [489]:
opt_k = 19
prediction = []
test_dataset = test_df.values
test_label_data = test_label_df.values
for x in range(len(test_dataset)):
    neighbours = getneighbours(train_dataset,test_dataset[x,:], False, opt_k)
    result = getresponse(neighbours)
    prediction.append(result)
precision, recall, F1_score, accuracy = get_performance_measure(test_label_data,prediction)
print("Precision=%f" % precision, "Recall=%f" %recall, "F1-score=%f" %F1_score,"accuracy=%f" %accuracy)

Precision=0.151515 Recall=0.118812 F1-score=0.133185 accuracy=21.900000


## Dropping missing values

In [471]:
opt_k = 19
prediction = []
test_dataset = test_df.values
test_label_data = test_label_df.values
for x in range(len(test_dataset)):
    neighbours = getneighbours(train_dataset,test_dataset[x,:], False, opt_k)
    result = getresponse(neighbours)
    prediction.append(result)
precision, recall, F1_score, accuracy = get_performance_measure(test_label_data,prediction)
print("Precision=%f" % precision, "Recall=%f" %recall, "F1-score=%f" %F1_score,"accuracy=%f" %accuracy)

Precision=0.286017 Recall=0.267327 F1-score=0.276356 accuracy=29.300000


In [ ]:
#create_plot(train_df, title="Training Data")
prediction